In [6]:
from importlib import reload
reload(io)
reload(info)
reload(base)
reload(aux)

<module 'src.auxiliary_functions' from '/home/ugur/python-projects/cefis2/src/auxiliary_functions.py'>

In [1]:
import pandas as pd
import numpy as np
import src.in_out as io
import src.info as info
import src.auxiliary_functions as aux
import src.condition as con
import src.base as base
from itertools import combinations
from src.spread import get_spread

## Veri Hazırlama

In [2]:
#Verilere ait path'ler alınıyor
path_list=io.get_path('data')

In [3]:
#Okunacak sütun adları
cols= ["Time","BID price","ASK price"]
#Tüm veriler okunuyor
master_data=io.read_allData(path_list, cols=cols)
master_data.columns=['time','bid_price','ask_price','name','date']

In [4]:
#date sütunu Timestamp'e dönüştürülüyor ve tarih ve saat bilgisi birleştiriliyor
master_data['date']=pd.to_datetime(master_data['date'], errors='coerce')
master_data['date'] += pd.to_timedelta(master_data.pop('time').astype(str))

In [5]:
#zaman dilimi ekleniyor
master_data['time_period']=master_data.date.dt.hour

## Mid Price

In [6]:
# mid price hesaplanıyor
master_data['mid_price']=base.average_of_series(master_data.bid_price,master_data.ask_price)

In [7]:
# spread hesaplama için master_data, uygun bir forma dönüştürülüyor
df_mid_price=master_data.pivot(index='date', columns='name', values='mid_price')

## Spread
<img src="img/spread_flowchart.png">

In [8]:
values={'a_PNLTICK':1,
'a_TICKSIZE':1,
'b_PNLTICK':1,
'b_TICKSIZE':1}

In [9]:
# pair'lar alınıyor
pairs=combinations(df_mid_price.columns, 2)

In [10]:
#spread hesaplanıyor
spread_list=[
    base.groupby_date_time(df_mid_price.loc[:,pair]).apply(lambda x: get_spread(x,values))\
        for pair in pairs]
df_spread=pd.concat(spread_list,axis=1).droplevel([0,1])

In [11]:
#change alınıyor
df_change=base.groupby_date_time(df_spread).apply(aux.find_change)
df_change=df_change.droplevel([0,1])

In [153]:
df_change

,6AU8_6BU8,6AU8_6CU8,6BU8_6CU8
date,,,
2018-09-06 00:00:04,-0.000109,NaN,1.090000e-04
2018-09-06 00:00:05,0.000000,NaN,0.000000e+00
2018-09-06 00:00:06,0.000000,NaN,0.000000e+00
2018-09-06 00:00:07,0.000000,NaN,0.000000e+00
2018-09-06 00:00:08,0.000000,NaN,0.000000e+00
...,...,...,...
2018-09-07 23:58:59,-0.000012,-0.000012,1.110223e-16
2018-09-07 23:59:00,0.000000,NaN,0.000000e+00
2018-09-07 23:59:01,0.000000,NaN,0.000000e+00


In [156]:
df= df_change['6AU8_6BU8']

In [162]:
sonuc= pd.DataFrame([df,x]).T

In [168]:
df_change[pd.Timestamp]

0.0

In [160]:
x=(df*df.shift()<0).cumsum()+1

In [164]:
sonuc.to_excel('sonuc.xlsx')

In [123]:
x.reset_index(inplace=True)

In [133]:
x.groupby(['sign']).transform('sum')

,urun,s
0,-0.000109,-0.001199
1,-0.000109,-0.001199
2,-0.000109,-0.001199
3,-0.000109,-0.001199
4,-0.000109,-0.001199
...,...,...
78561,-0.000012,-0.000046
78562,-0.000012,-0.000046
78563,-0.000012,-0.000046
78564,-0.000012,-0.000046


In [154]:
x

,date,urun,sign,s
0,2018-09-06 00:00:04,-0.000109,1.0,-0.000109
1,2018-09-06 00:00:05,0.000000,1.0,-0.000109
2,2018-09-06 00:00:06,0.000000,1.0,-0.000109
3,2018-09-06 00:00:07,0.000000,1.0,-0.000109
4,2018-09-06 00:00:08,0.000000,1.0,-0.000109
...,...,...,...,...
78561,2018-09-07 23:58:59,-0.000012,5874.0,-0.000012
78562,2018-09-07 23:59:00,0.000000,5874.0,-0.000012
78563,2018-09-07 23:59:01,0.000000,5874.0,-0.000012
78564,2018-09-07 23:59:02,0.000000,5874.0,-0.000012


In [150]:
y

0       2018-09-06 00:00:04
11      2018-09-06 00:00:15
13      2018-09-06 00:00:17
35      2018-09-06 00:00:39
41      2018-09-06 00:00:45
                ...        
78554   2018-09-07 23:58:52
78556   2018-09-07 23:58:54
78559   2018-09-07 23:58:57
78561   2018-09-07 23:58:59
78565   2018-09-07 23:59:03
Name: date, Length: 11758, dtype: datetime64[ns]

In [148]:
zz

0            NaT
11      00:00:11
13      00:00:02
35      00:00:22
41      00:00:06
          ...   
78554   00:00:01
78556   00:00:02
78559   00:00:03
78561   00:00:02
78565   00:00:04
Name: date, Length: 11758, dtype: timedelta64[ns]

In [47]:
def find_duramp(data):
    """ (+),(-) yönde ya da sabit bir dalganın süresini hesaplar
    Parameters:
        data (pandas.Series): change
    Returns:
        pd.Series: Her bir yönde gerçekleşen değişimin süresi
    """
    data = data.dropna()
    size = len(data)
    dur_list = size*[None]
    amp_list = size*[None]
    sign_negativ =0
    sign_positive=0
    amplitude = 0
    #sign_zero=0
    for i in range(0,size+1):
        try:
            change = data.iat[i]            
        except:
            if sign_positive>0:
                dur_list[i-1] = sign_positive
                amp_list[i-1] = amplitude
            else:
                if sign_negativ>0:
                    dur_list[i-1] = sign_negativ
                else:
                    continue
                
        if change<0:
            if (sign_positive==0 and sign_negativ==0 ) or (sign_positive==0 and sign_negativ>0):
                sign_negativ+=1
                amplitude +=change
            if sign_positive>0:
                amp_list[i-1]=amplitude
                dur_list[i-1]=sign_positive
                sign_positive=0
                sign_negativ+=1
                amplitude=change

        elif change>0:
            if (sign_negativ==0 and sign_positive==0 ) or (sign_negativ==0 and sign_positive>0):
                amplitude+=change
                sign_positive+=1
            if sign_negativ>0:
                amp_list[i-1]=amplitude
                dur_list[i-1]=sign_negativ
                sign_negativ=0
                sign_positive+=1
                amplitude=change
        else:
            if sign_negativ>0:
                sign_negativ+=1
            elif sign_positive>0:
                sign_positive+=1
            else:
                continue
            
    series_dur = pd.Series(data=dur_list,name='duration',index=data.index).dropna()
    series_amp = pd.Series(data=amp_list,name='amplitude',index=data.index).dropna()
    print(pd.DataFrame(data=[series_dur,series_amp]).transpose())
    return pd.DataFrame(data=[series_dur,series_amp]).transpose()